# 无限接近
<div>
    <img src="无限接近.png"/>
</div>

# 无限远离
<div>
    <img src="无限远离.png"/>
</div>

# 有交集
<div>
    <img src="有交集.png"/>
</div>

# GIoU Loss
<div>
      <img src="GIoULoss.png" />
</div>

* GIoU的值域范围是：(-1, +1]
* GIoULoss = 1 - GIoU
* GIoULoss的值域范围是：[0, 2)

In [17]:
import torch 
import torch.nn as nn

In [30]:
A = torch.tensor([
    [1, 1, 10, 10],
    [5, 5, 8, 8],
    [1, 1, 10, 10]
]).float()

B = torch.tensor([
    [2, 3, 11, 11],
    [6, 6, 9, 9],
    [20, 20, 30, 30]
]).float()
A.shape, B.shape

(torch.Size([3, 4]), torch.Size([3, 4]))

In [38]:
ax, ay, ar, ab = A[:, 0], A[:, 1], A[:, 2], A[:, 3]
bx, by, br, bb = B[:, 0], B[:, 1], B[:, 2], B[:, 3]
cross_xmin = torch.max(ax, bx)
cross_ymin = torch.max(ay, by)
cross_xmax = torch.min(ar, br)
cross_ymax = torch.min(ab, bb)
cross_width = (cross_xmax - cross_xmin + 1).clamp(0)
cross_height = (cross_ymax - cross_ymin + 1).clamp(0)
cross_area = cross_width * cross_height
a_area = (ar - ax + 1) * (ab - ay + 1)
b_area = (br - bx + 1) * (bb - by + 1)
union_area = a_area + b_area - cross_area
iou = cross_area / union_area

# C指的是AB最小的外包框
c_xmin = torch.min(ax, bx)
c_ymin = torch.min(ay, by)
c_xmax = torch.max(ar, br)
c_ymax = torch.max(ab, bb)
c_area = (c_xmax - c_xmin + 1) * (c_ymax - c_ymin + 1)
giou = iou - (c_area - union_area) / c_area  # 取值范围是：(-1, +1]
giouloss = 1 - giou # 取值范围是：[0, +2)
giouloss.mean()

tensor(0.9526)

In [2]:
import torch
import torch.nn as nn

In [9]:
A = torch.tensor([
    [1, 1, 10, 10],
    [5, 5, 8, 8],
    [1, 1, 10, 10]
]).float()

B = torch.tensor([
    [2, 3, 11, 11],
    [6, 6, 9, 9],
    [20, 20, 30, 30]
]).float()

In [10]:
ax, ay, ar, ab = A[:, 0], A[:, 1], A[:, 2], A[:, 3]
bx, by, br, bb = B[:, 0], B[:, 1], B[:, 2], B[:, 3]
xmax = torch.min(ar, br)
ymax = torch.min(ab, bb)
xmin = torch.max(ax, bx)
ymin = torch.max(ay, by)
cross_width = (xmax - xmin + 1).clamp(0)
cross_height = (ymax - ymin + 1).clamp(0)
cross = cross_width * cross_height
cross

tensor([72.,  9.,  0.])

In [11]:
union = (ar - ax + 1) * (ab - ay + 1) + (br - bx + 1) * (bb - by + 1) - cross
union

tensor([118.,  23., 221.])

In [12]:
iou = cross / union
iou

tensor([0.6102, 0.3913, 0.0000])

In [13]:
cxmin = torch.min(ax, bx)
cymin = torch.min(ay, by)
cxmax = torch.max(ar, br)
cymax = torch.max(ab, bb)
c = (cxmax - cxmin + 1) * (cymax - cymin + 1)
c

tensor([121.,  25., 900.])

In [14]:
GIoULoss = 1 - (iou - (c - union) / c)
GIoULoss.sum() / 3 / 2

tensor(0.4763)

In [41]:
def GIoULoss(A, B):
    num_bbox = A.size(0)
    ax, ay, ar, ab = A[:, 0], A[:, 1], A[:, 2], A[:, 3]
    bx, by, br, bb = B[:, 0], B[:, 1], B[:, 2], B[:, 3]
    xmax = torch.min(ar, br)
    ymax = torch.min(ab, bb)
    xmin = torch.max(ax, bx)
    ymin = torch.max(ay, by)
    cross_width = (xmax - xmin + 1).clamp(0)
    cross_height = (ymax - ymin + 1).clamp(0)
    cross = cross_width * cross_height
    union = (ar - ax + 1) * (ab - ay + 1) + (br - bx + 1) * (bb - by + 1) - cross
    iou = cross / union
    cxmin = torch.min(ax, bx)
    cymin = torch.min(ay, by)
    cxmax = torch.max(ar, br)
    cymax = torch.max(ab, bb)
    c = (cxmax - cxmin + 1) * (cymax - cymin + 1)
    return (1 - (iou - (c - union) / c)).mean()

In [42]:
GIoULoss(A, B)

tensor(0.9526)

In [10]:
GIoULoss(A, A)

tensor(0.)